In [1]:
from PIL import Image
import os
import glob
from pathlib import Path
import json
import os, shutil


def zoom_center(image, zoom_factor):
    # Get image dimensions
    width, height = image.size

    # Calculate the center coordinates
    center_x, center_y = width // 2, height // 2

    # Calculate the size of the zoomed-in region
    zoomed_width = int(width * zoom_factor)
    zoomed_height = int(height * zoom_factor)

    # Calculate the coordinates for cropping
    crop_x1 = max(center_x - zoomed_width // 2, 0)
    crop_x2 = min(center_x + zoomed_width // 2, width)
    crop_y1 = max(center_y - zoomed_height // 2, 0)
    crop_y2 = min(center_y + zoomed_height // 2, height)

    # Crop the region around the center
    cropped_region = image.crop((crop_x1, crop_y1, crop_x2, crop_y2))

    # Resize the cropped region back to the original dimensions
    zoomed_image = cropped_region.resize((width, height))

    return zoomed_image

def add_background(background_path, transparent_image_path, output_path, zoom, zoom_factor=0.8):
    try:
        # Open the background image
        background = Image.open(background_path)

        # Open the transparent image
        transparent_image = Image.open(transparent_image_path)

        if zoom:
            # Specify the zoom factor (adjust as needed)

            # Zoom into the center of the image
            transparent_image = zoom_center(transparent_image, zoom_factor)

        # Resize the transparent image to match the background size
        # transparent_image = transparent_image.resize(background.size, Image.ANTIALIAS)
        # background = background.resize(transparent_image.size, Image.ANTIALIAS)
        background = background.resize(transparent_image.size, Image.Resampling.LANCZOS)

        # Composite the images
        result = Image.alpha_composite(background.convert('RGBA'), transparent_image)

        final_result = result.convert('RGB')
        # Save the result
                
        final_result.save(output_path, format='PNG')
    except:
        return

In [7]:
BACKGROUND_PATH = r"background.png"
ICONS_FOLDER = r"icons"
FINAL_FOLDER = r"icons_processed"

os.makedirs(FINAL_FOLDER, exist_ok=True)

names_dict = json.load(open("convert_names.txt", "r"))
reversed_dict = {value: key for key, value in names_dict.items()}
for full_path in glob.glob(f"{ICONS_FOLDER}/*.png"):
    original_filename = Path(full_path).name
    new_filename = reversed_dict.get(original_filename)
    if not new_filename:
        exception_list = ["empty", "coin", "metal", "wood"]
        if any(substring in original_filename for substring in exception_list):
            new_filename = original_filename
        else:
            new_filename = f"zzz{original_filename}"
    output_path = os.path.join(FINAL_FOLDER, new_filename)
    add_background(BACKGROUND_PATH, full_path, output_path, zoom=True, zoom_factor=0.95)
    if new_filename.startswith("Mega_"):
        custom_path = Path(output_path)
        filename_without_extension = custom_path.stem
        custom_path = custom_path.with_name(f"{filename_without_extension}_zoomed{custom_path.suffix}")
        add_background(BACKGROUND_PATH, full_path, custom_path, zoom=True)